In [4]:
import os
import pandas as pd
import patsy
import numpy as np

In [5]:
if not os.getcwd().endswith('assignment3'):
    os.chdir(os.path.join('assignment3'))
# read the file
loans = pd.read_csv('loans.csv')

# rename problematic coulmns
loans = loans.rename(columns={'credit.policy': 'creditPolicy',
                              'int.rate': 'intRate',
                              'log.annual.inc': 'logAnnualInc',
                              'days.with.cr.line': 'daysWithCrLine',
                              'revol.bal': 'revolBal',
                              'revol.util': 'revolUtil',
                              'inq.last.6mths': 'inqLast6mths',
                              'delinq.2yrs': 'delinq2yrs',
                              'pub.rec': 'pubRec',
                              'not.fully.paid': 'notFullyPaid'})
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
creditPolicy      9578 non-null int64
purpose           9578 non-null object
intRate           9578 non-null float64
installment       9578 non-null float64
logAnnualInc      9574 non-null float64
dti               9578 non-null float64
fico              9578 non-null int64
daysWithCrLine    9549 non-null float64
revolBal          9578 non-null int64
revolUtil         9516 non-null float64
inqLast6mths      9549 non-null float64
delinq2yrs        9549 non-null float64
pubRec            9549 non-null float64
notFullyPaid      9578 non-null int64
dtypes: float64(9), int64(4), object(1)
memory usage: 1.0+ MB


### Problem 1.1 - What proportion of the loans in the dataset were not paid in full? 

In [6]:
paid = (loans['notFullyPaid'] == 1).sum()
total = loans['notFullyPaid'].count()
paid/total

0.16005429108373356

Althernativley - and more simple

In [7]:
loans['notFullyPaid'].mean()

0.16005429108373356

### Problem 1.2 - find the variables with missing data

In [8]:
loans.isnull().sum()

creditPolicy       0
purpose            0
intRate            0
installment        0
logAnnualInc       4
dti                0
fico               0
daysWithCrLine    29
revolBal           0
revolUtil         62
inqLast6mths      29
delinq2yrs        29
pubRec            29
notFullyPaid       0
dtype: int64

### Problem 1.4 - Prepare data
In this case, read pre-imputed data

In [9]:
loans_imputed = pd.read_csv('loans_imputed.csv')

# rename problematic coulmns
loans_imputed = loans_imputed.rename(columns={'credit.policy': 'creditPolicy',
                              'int.rate': 'intRate',
                              'log.annual.inc': 'logAnnualInc',
                              'days.with.cr.line': 'daysWithCrLine',
                              'revol.bal': 'revolBal',
                              'revol.util': 'revolUtil',
                              'inq.last.6mths': 'inqLast6mths',
                              'delinq.2yrs': 'delinq2yrs',
                              'pub.rec': 'pubRec',
                              'not.fully.paid': 'notFullyPaid'})
loans_imputed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9578 entries, 0 to 9577
Data columns (total 14 columns):
creditPolicy      9578 non-null int64
purpose           9578 non-null object
intRate           9578 non-null float64
installment       9578 non-null float64
logAnnualInc      9578 non-null float64
dti               9578 non-null float64
fico              9578 non-null int64
daysWithCrLine    9578 non-null float64
revolBal          9578 non-null int64
revolUtil         9578 non-null float64
inqLast6mths      9578 non-null int64
delinq2yrs        9578 non-null int64
pubRec            9578 non-null int64
notFullyPaid      9578 non-null int64
dtypes: float64(6), int64(7), object(1)
memory usage: 1.0+ MB


### Problem 2.1
Split by 70% and use logistic regression trained on the training set to predict the dependent variable not.fully.paid using all the independent variables.

In [10]:
np.random.seed(248)
from sklearn.model_selection import train_test_split
loans_train, loans_test = train_test_split(loans_imputed, test_size=0.3)

In [11]:
from statsmodels.discrete.discrete_model import Logit
y, X = patsy.dmatrices('notFullyPaid ~ creditPolicy + C(purpose) + '
                       'intRate + installment + logAnnualInc + '
                       'daysWithCrLine + revolBal + dti + fico + '
                       'revolUtil + inqLast6mths + delinq2yrs + '
                       'pubRec', loans_train, return_type='dataframe')

mod1 = Logit(y, X).fit()
mod1.summary2()

Optimization terminated successfully.
         Current function value: 0.416406
         Iterations 6


<table class="simpletable">
<tr>
        <td>Model:</td>              <td>Logit</td>      <td>Pseudo R-squared:</td>    <td>0.067</td>  
</tr>
<tr>
  <td>Dependent Variable:</td>   <td>notFullyPaid</td>         <td>AIC:</td>         <td>5621.1757</td>
</tr>
<tr>
         <td>Date:</td>        <td>2017-03-27 16:12</td>       <td>BIC:</td>         <td>5750.5745</td>
</tr>
<tr>
   <td>No. Observations:</td>        <td>6704</td>        <td>Log-Likelihood:</td>    <td>-2791.6</td> 
</tr>
<tr>
       <td>Df Model:</td>             <td>18</td>            <td>LL-Null:</td>        <td>-2992.5</td> 
</tr>
<tr>
     <td>Df Residuals:</td>          <td>6685</td>         <td>LLR p-value:</td>    <td>3.7987e-74</td>
</tr>
<tr>
      <td>Converged:</td>           <td>1.0000</td>           <td>Scale:</td>         <td>1.0000</td>  
</tr>
<tr>
    <td>No. Iterations:</td>        <td>6.0000</td>              <td></td>               <td></td>     
</tr>
</table>
<table class="simpletable">
<tr>
                  <td></td>                  <th>Coef.</th>  <th>Std.Err.</th>    <th>z</th>     <th>P>|z|</th> <th>[0.025</th>  <th>0.975]</th> 
</tr>
<tr>
  <th>Intercept</th>                        <td>9.2463</td>   <td>1.5391</td>  <td>6.0076</td>  <td>0.0000</td> <td>6.2297</td>  <td>12.2629</td>
</tr>
<tr>
  <th>C(purpose)[T.credit_card]</th>        <td>-0.5693</td>  <td>0.1275</td>  <td>-4.4663</td> <td>0.0000</td> <td>-0.8191</td> <td>-0.3194</td>
</tr>
<tr>
  <th>C(purpose)[T.debt_consolidation]</th> <td>-0.3994</td>  <td>0.0911</td>  <td>-4.3846</td> <td>0.0000</td> <td>-0.5780</td> <td>-0.2209</td>
</tr>
<tr>
  <th>C(purpose)[T.educational]</th>        <td>-0.0288</td>  <td>0.1786</td>  <td>-0.1613</td> <td>0.8719</td> <td>-0.3789</td> <td>0.3213</td> 
</tr>
<tr>
  <th>C(purpose)[T.home_improvement]</th>   <td>0.1253</td>   <td>0.1478</td>  <td>0.8477</td>  <td>0.3966</td> <td>-0.1644</td> <td>0.4150</td> 
</tr>
<tr>
  <th>C(purpose)[T.major_purchase]</th>     <td>-0.4789</td>  <td>0.2036</td>  <td>-2.3524</td> <td>0.0187</td> <td>-0.8780</td> <td>-0.0799</td>
</tr>
<tr>
  <th>C(purpose)[T.small_business]</th>     <td>0.4856</td>   <td>0.1412</td>  <td>3.4378</td>  <td>0.0006</td> <td>0.2087</td>  <td>0.7624</td> 
</tr>
<tr>
  <th>creditPolicy</th>                     <td>-0.3217</td>  <td>0.0996</td>  <td>-3.2300</td> <td>0.0012</td> <td>-0.5168</td> <td>-0.1265</td>
</tr>
<tr>
  <th>intRate</th>                          <td>-0.0754</td>  <td>2.0912</td>  <td>-0.0361</td> <td>0.9712</td> <td>-4.1741</td> <td>4.0232</td> 
</tr>
<tr>
  <th>installment</th>                      <td>0.0011</td>   <td>0.0002</td>  <td>5.3992</td>  <td>0.0000</td> <td>0.0007</td>  <td>0.0016</td> 
</tr>
<tr>
  <th>logAnnualInc</th>                     <td>-0.4314</td>  <td>0.0707</td>  <td>-6.1044</td> <td>0.0000</td> <td>-0.5700</td> <td>-0.2929</td>
</tr>
<tr>
  <th>daysWithCrLine</th>                   <td>0.0000</td>   <td>0.0000</td>  <td>0.8964</td>  <td>0.3700</td> <td>-0.0000</td> <td>0.0000</td> 
</tr>
<tr>
  <th>revolBal</th>                         <td>0.0000</td>   <td>0.0000</td>  <td>2.5978</td>  <td>0.0094</td> <td>0.0000</td>  <td>0.0000</td> 
</tr>
<tr>
  <th>dti</th>                              <td>0.0015</td>   <td>0.0055</td>  <td>0.2741</td>  <td>0.7840</td> <td>-0.0093</td> <td>0.0123</td> 
</tr>
<tr>
  <th>fico</th>                             <td>-0.0093</td>  <td>0.0017</td>  <td>-5.4682</td> <td>0.0000</td> <td>-0.0126</td> <td>-0.0060</td>
</tr>
<tr>
  <th>revolUtil</th>                        <td>0.0032</td>   <td>0.0015</td>  <td>2.1138</td>  <td>0.0345</td> <td>0.0002</td>  <td>0.0062</td> 
</tr>
<tr>
  <th>inqLast6mths</th>                     <td>0.0919</td>   <td>0.0163</td>  <td>5.6372</td>  <td>0.0000</td> <td>0.0600</td>  <td>0.1239</td> 
</tr>
<tr>
  <th>delinq2yrs</th>                       <td>-0.1124</td>  <td>0.0677</td>  <td>-1.6592</td> <td>0.0971</td> <td>-0.2452</td> <td>0.0204</td> 
</tr>
<tr>
  <th>pubRec</th>          

### Problem 2.2
A has FICO credit score 700 while the borrower in Application B has FICO credit score 710. What is the value of Logit(A) - Logit(B)?

In [12]:
result = (700-710) * -0.009
result

0.09

let O(A) be the odds of loan A not being paid back in full,  What is the value of O(A)/O(B)? Reminder: $$Odds = \frac {F(x)}{1-F(x)}= exp(logit)$$

In [13]:
np.exp(result)

1.0941742837052104

### Problem 2.3 - Prediction Models

In [14]:
y_test, X_test = patsy.dmatrices('notFullyPaid ~ creditPolicy + C(purpose) + '
                       'intRate + installment + logAnnualInc + '
                       'daysWithCrLine + revolBal + dti + fico + '
                       'revolUtil + inqLast6mths + delinq2yrs + '
                       'pubRec', loans_test, return_type='dataframe')
precitions_mod1 = mod1.predict(exog=X_test)

In [15]:
from sklearn.metrics import confusion_matrix

thresh = 0.5
con_matx = confusion_matrix(loans_test['notFullyPaid'].values, precitions_mod1 > thresh).astype(np.int)
con_matx

array([[2423,   18],
       [ 422,   11]])

In [16]:
TP = con_matx[0, 1]
TN = con_matx[0, 0]
FP = con_matx[1, 1]
FN = con_matx[1, 0]

What is the accuracy of the logistic regression model?

In [17]:
(TP + TN) / con_matx.sum()

0.8493389004871259

What is the accuracy of the baseline model?

Answer: the baseline model assumes that all loans will be fully paid.

In [18]:
((1-loans_test['notFullyPaid'].values).sum())/loans_test['notFullyPaid'].count()

0.8493389004871259

### Problem 2.4 - compute the test set AUC

In [19]:
from sklearn.metrics import roc_auc_score as auc
auc(y_true=loans_test['notFullyPaid'], y_score=precitions_mod1)

0.68088079602404272

### Problem 3.1 - "smart" baseline
Using the training set, build a bivariate logistic regression model that predicts the dependent variable not.fully.paid using only the variable int.rate

In [20]:
y_baseline, X_baseline = patsy.dmatrices('notFullyPaid ~ intRate', loans_train, return_type='dataframe')

baseline_mod = Logit(y_baseline, X_baseline).fit()
baseline_mod.summary2()

Optimization terminated successfully.
         Current function value: 0.435409
         Iterations 6


<table class="simpletable">
<tr>
        <td>Model:</td>              <td>Logit</td>      <td>Pseudo R-squared:</td>    <td>0.025</td>  
</tr>
<tr>
  <td>Dependent Variable:</td>   <td>notFullyPaid</td>         <td>AIC:</td>         <td>5841.9645</td>
</tr>
<tr>
         <td>Date:</td>        <td>2017-03-27 16:12</td>       <td>BIC:</td>         <td>5855.5854</td>
</tr>
<tr>
   <td>No. Observations:</td>        <td>6704</td>        <td>Log-Likelihood:</td>    <td>-2919.0</td> 
</tr>
<tr>
       <td>Df Model:</td>              <td>1</td>            <td>LL-Null:</td>        <td>-2992.5</td> 
</tr>
<tr>
     <td>Df Residuals:</td>          <td>6702</td>         <td>LLR p-value:</td>    <td>7.6822e-34</td>
</tr>
<tr>
      <td>Converged:</td>           <td>1.0000</td>           <td>Scale:</td>         <td>1.0000</td>  
</tr>
<tr>
    <td>No. Iterations:</td>        <td>6.0000</td>              <td></td>               <td></td>     
</tr>
</table>
<table class="simpletable">
<tr>
      <td></td>       <th>Coef.</th>  <th>Std.Err.</th>     <th>z</th>     <th>P>|z|</th> <th>[0.025</th>  <th>0.975]</th> 
</tr>
<tr>
  <th>Intercept</th> <td>-3.5319</td>  <td>0.1667</td>  <td>-21.1842</td> <td>0.0000</td> <td>-3.8587</td> <td>-3.2051</td>
</tr>
<tr>
  <th>intRate</th>   <td>15.0994</td>  <td>1.2615</td>   <td>11.9693</td> <td>0.0000</td> <td>12.6269</td> <td>17.5719</td>
</tr>
</table>

### Problem 3.2
Make test set predictions for the bivariate model. What is the highest predicted probability of a loan not being paid in full on the testing set?

In [21]:
y_baseline_test, X_baseline_test = patsy.dmatrices('notFullyPaid ~ intRate', loans_test, return_type='dataframe')
pred_baseline_mod = baseline_mod.predict(exog=X_baseline_test)

In [22]:
pred_baseline_mod.max()

0.41840511188946128

With a logistic regression cutoff of 0.5, how many loans would be predicted as not being paid in full on the testing set?

In [23]:
(pred_baseline_mod > thresh).sum()

0

### Problem 3.3 - Calculate AUC for this baseline

In [24]:
auc(y_true=loans_test['notFullyPaid'], y_score=pred_baseline_mod)

0.64302196975646031

 ### Problem 4.1 - Computing the Profitability of an Investment 

In [25]:
10 * np.exp(0.06*3)  # equals to the amount payed back

11.972173631218102

 ### Problem 5.1 - A Simple Investment Strategy
 Enter the following commands in your R console to create this new variable:

test\\$profit = exp(test$int.rate*3) - 1

test\\$profit[test$not.fully.paid == 1] = -1

In [26]:
loans_test['profit'] = np.exp(loans_test['intRate']*3) - 1

C:\Users\tsivi\Miniconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
loans_test[['intRate', 'profit']].head()

,intRate,profit
7343,0.1025,0.360021
5310,0.0894,0.307609
1311,0.0832,0.283512
3575,0.1158,0.415383
2267,0.1221,0.442388


What is the maximum profit of a $10 investment in any loan in the testing set?

In [28]:
loans_test['profit'].max() * 10  # 10 dollars

8.8947686546753317

### Problem 6.1 
irst, use the subset() function to build a data frame called highInterest consisting of the test set loans with an interest rate of at least 15%

In [29]:
maxRisk = loans_test.query('intRate >= 0.15')

What is the average profit of a $1 investment in one of these high-interest loans 

In [44]:
maxRisk['profit'].mean()

0.64386106945522836

This is considerably larger than the number they get on Edx (about 0.22). I've tried only the highest 100:

In [37]:
maxRisk['profit'].nlargest(100).describe()

count    100.000000
mean       0.760350
std        0.054865
min        0.684890
25%        0.714463
50%        0.745604
75%        0.793376
max        0.889477
Name: profit, dtype: float64

and the mean seem to be even higher.

Q. What proportion of the high-interest loans were not paid back in full?

In [39]:
maxRisk['notFullyPaid'].mean()

0.2585812356979405

### Problem 6.2
I'm a little confused about this problem. I'll just skip it.